In [5]:
from langchain.schema import Document

In [6]:
from langchain.vectorstores import Chroma

In [7]:
from langchain_ollama.embeddings import OllamaEmbeddings

In [ ]:
doc1 = Document(page_content="Virat Kohli (Batsman, Former Captain): Scored 8 centuries in 2024, ranking among the top Test and ODI batsmen globally.",
                metadata={"role":"Bowler"})

In [9]:
doc2 = Document(page_content="Rohit Sharma (Batsman, ODI/Test Captain): Led India to the 2024 T20 World Cup victory, ranked third in ICC ODI batting rankings",
                metadata={"role":"Batsman, ODI/Test Captain"})

In [10]:
doc3 = Document(page_content="Jasprit Bumrah (Bowler): Top-ranked Test bowler, leading India’s pace attack with 6/70 in the 2025 England Test series.",
                metadata={"role":"(Bowler)"})

In [11]:
docs=[doc1,doc2,doc3]

In [12]:
vector_store =Chroma(
   embedding_function=OllamaEmbeddings(model="granite-embedding:latest"),
   persist_directory="my_chroma_db",
   collection_name="sample"
)

C:\Users\SR43993\AppData\Local\Temp\ipykernel_22244\3164047253.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store =Chroma(


In [20]:
vector_store.similarity_search(
    query="Who among are the top bowlers",
    k=2
)

[Document(metadata={'role': '(Bowler)'}, page_content='Jasprit Bumrah (Bowler): Top-ranked Test bowler, leading India’s pace attack with 6/70 in the 2025 England Test series.'),
 Document(metadata={'role': '(Bowler)'}, page_content='Jasprit Bumrah (Bowler): Top-ranked Test bowler, leading India’s pace attack with 6/70 in the 2025 England Test series.')]

In [16]:
vector_store.add_documents(docs)

['fe6c3453-975e-43a8-a4be-b4da64e5492c',
 'b087ca0b-2154-4378-96a0-ccf4a087a2be',
 '802b88d2-8849-4fa7-83ea-0f848489bd9e']

In [17]:
vector_store.get(include=["embeddings","documents","metadatas"])

{'ids': ['d2277f2b-f437-4a9e-874f-7f5daa6fbba3',
  '6b0cfb0a-7a61-4a66-af89-a3cf562a60b4',
  '61b1bdac-991f-466b-bb99-f301794d2cf0',
  'fe6c3453-975e-43a8-a4be-b4da64e5492c',
  'b087ca0b-2154-4378-96a0-ccf4a087a2be',
  '802b88d2-8849-4fa7-83ea-0f848489bd9e'],
 'embeddings': array([[-0.06681802,  0.0433434 , -0.03207217, ..., -0.00888443,
         -0.20114835,  0.04152339],
        [-0.01794345, -0.01206979, -0.03552095, ...,  0.02271054,
         -0.16851369,  0.06533858],
        [-0.05643566,  0.01129285, -0.04525478, ...,  0.02857568,
         -0.14442395,  0.06133389],
        [-0.06681802,  0.0433434 , -0.03207217, ..., -0.00888443,
         -0.20114835,  0.04152339],
        [-0.01794345, -0.01206979, -0.03552095, ...,  0.02271054,
         -0.16851369,  0.06533858],
        [-0.05643566,  0.01129285, -0.04525478, ...,  0.02857568,
         -0.14442395,  0.06133389]], shape=(6, 384)),
 'documents': ['Virat Kohli (Batsman, Former Captain): Scored 8 centuries in 2024, ranking among